In [1]:
import IPython.display as ipd
import torch
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols_zh
from text import text_to_sequence
import soundfile as sf
from chinese import get_phoneme_from_text, get_pinyin_from_text


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners, phoneme=True)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file("./configs/csmsc.json")
net_g = SynthesizerTrn(
    len(symbols_zh),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)#.cuda()
_ = net_g.eval()


MODEL_CHECKPOINT = "./logs/csmsc/G_408000.pth"
TEXT = "早上好"

_ = utils.load_checkpoint(MODEL_CHECKPOINT, net_g, None)

phonemes = get_pinyin_from_text(TEXT)
print(phonemes)

stn_tst = get_text(phonemes, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    net_g.to(x_tst.device)
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
sf.write('chinese.wav', audio, hps.data.sampling_rate)

/home/jiyuyu/miniconda3/envs/tts/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/jiyuyu/workspace/vits/utils.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_on

INFO:root:Loaded checkpoint './logs/csmsc/G_343000.pth' (iteration 839)


Building prefix dict from the default dictionary ...


DEBUG:jieba:Building prefix dict from the default dictionary ...


Loading model from cache /tmp/jieba.cache


...

Loading model cost 0.524 seconds.


DEBUG:jieba:Loading model cost 0.524 seconds.


Prefix dict has been built successfully.


DEBUG:jieba:Prefix dict has been built successfully.


KeyboardInterrupt: 

In [5]:
from funasr import AutoModel

model = AutoModel(
    model="paraformer-zh",
)

res = model.generate(input="chinese.wav")
print(res)

funasr version: 1.1.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.1.6


2024-09-20 04:05:55,525 - modelscope - WARNING - Using branch: master as version is unstable, use with caution

/home/jiyuyu/miniconda3/envs/tts/lib/python3.12/site-packages/funasr/models/paraformer/model.py:251: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(False):



rtf_avg: 0.434: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it]

[{'key': 'chinese', 'text': '你 好 这 里 是 新 加 坡 国 立 大 学', 'timestamp': [[330, 510], [510, 750], [770, 910], [910, 1030], [1030, 1230], [1230, 1370], [1370, 1610], [1610, 1790], [1790, 1890], [1890, 2090], [2090, 2250], [2250, 2485]]}]
